In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import pandas as pd
import healpy as hp
from astropy.io import fits
import pyccl as ccl

from tqdm import tqdm #for timing

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("seaborn-notebook")

from IPython.display import HTML
from IPython.display import display

import sys
sys.path.insert(1, '../src/')
from lib import *

In this notebook, we all the relevant plots.

**LIST PLOTS**

#### Load saved $\tilde{C}_{\ell}$ and use them for comparison

In [10]:
#parameters
fs = 20
fsize = (10, 7)

NSIDE = 1024
lmin = 100; 
LMAX = 3 * NSIDE - 1
ell = np.arange(0, LMAX, 1)
ELL = np.logspace(0, np.log10(NSIDE), 10) #for binning

NMOCKS = 1000

In [11]:
#read in coupling matrix
#M = np.load("../dat/img-sys-couplingmatrix.npy")
exp_names = np.array(['A', 'B', 'C', 'D', 'E', 'F'])
experiments = {}

#dict to store values
for name in exp_names:
    experiments[name] = {}

#load cls from memory

for key in experiments:
    experiments[key]['cls_obs'] = np.load("../dat/pCls/1000mocks/pCls_" + key + ".npy")
    experiments[key]['CLs_obs'] = np.zeros((NMOCKS, ELL.shape[0] - 1))
    experiments[key]['pCLs_obs'] = np.zeros((NMOCKS, ELL.shape[0] - 1))
    
   #read in fsky and nl
    experiments[key]['nl'] = np.load("../dat/pCls/1000mocks/noise_window_" +key +".npy")
    experiments[key]['fsky'] = np.load("../dat/pCls/1000mocks/fsky_" +key +".npy")
    
#calculate pCls
for key in ['A', 'B', 'C']:
    experiments[key]['pcls_obs'] = (experiments[key]['cls_obs'] * experiments[key]['fsky'][:,np.newaxis]) + \
    experiments[key]['nl'][:,np.newaxis]
    
for key in ['D', 'E', 'F']:
    experiments[key]['pcls_obs'] = (experiments[key]['cls_obs'] + experiments[key]['nl'][:,np.newaxis]) * \
     experiments[key]['fsky'][:,np.newaxis]
    
#bin power spectrum
for key in experiments:
    for i in range(NMOCKS):
        ELL_mid, experiments[key]['CLs_obs'][i] = bin_mat(ell[:NSIDE], 
                                                           experiments[key]['cls_obs'][i][:NSIDE],
                                                           ELL) #Cls up to NSIDE is good 
        ELL_mid, experiments[key]['pCLs_obs'][i] = bin_mat(ell[:NSIDE], 
                                                           experiments[key]['pcls_obs'][i][:NSIDE],
                                                           ELL) #Cls up to NSIDE is good 

../src/lib.py:171: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  mat_t=mat_r_dr[slc]


In [12]:
#calculate cov matrix and cumulative SN
for key in experiments: 
    experiments[key]['covmat'] = np.cov(experiments[key]['pCLs_obs'].T)
    experiments[key]['corrcoeff'] = np.corrcoef(experiments[key]['pCLs_obs'].T)
    experiments[key]['SN_cum'] = SN_cum(experiments[key]['covmat'], ELL_mid, 
                                       experiments[key]['pCLs_obs'].mean(axis = 0), lmax = LMAX,
                                       use_hartlap=True, nsim = NMOCKS)
    
#set colours for models
experiments['A']['colour'] = 'black'
experiments['B']['colour'] = 'red'
experiments['C']['colour'] = 'green'
experiments['D']['colour'] = 'black'
experiments['E']['colour'] = 'red'
experiments['F']['colour'] = 'green'